In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import time
import matplotlib.pyplot as plt
import seaborn 
from matplotlib import animation

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1.数据处理

In [ ]:
data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data['ObservationDate'] = data['ObservationDate'].astype('datetime64')
data['Last Update'] = data['Last Update'].astype('datetime64')#改变特征的数据类型
data.info()

处理缺失值

In [ ]:
data.isnull().sum()

Province/State的缺失值太多，且不影响对国家/地区的数据可视化，所以考虑去掉该特征

In [ ]:
data = data.drop('Province/State',axis=1)
data.isnull().sum()

按ObservationDate分组

In [ ]:
data1 = data.groupby('ObservationDate')
type(data1)

2.数据可视化

In [ ]:
Date_list = []
Region_list = []
Confiremd_list = []
for ObservationDate ,group in data1:
    # print(ObservationDate)
    dt_obj = ObservationDate.to_pydatetime()
    date_str = dt_obj.strftime("%Y-%m-%d %H:%M:%S")#将ObservationDate类型由timestamp转换为string
    Date_list.append(date_str[:10])
    df= group.groupby('Country/Region').agg('sum').sort_values(by='Confirmed',ascending=False).reset_index()
    df_region = df['Country/Region'].values.tolist()
    df_confiremd = df['Confirmed'].values.tolist()
    Region_list.append(df_region)
    Confiremd_list.append(df_confiremd)

In [ ]:
def draw_chart(Region_list,Confiremd_list):
    Color_list = ['red','aquamarine', 'dodgerblue','orange','deepskyblue','seagreen','gold','darkviolet','yellow','black']
    fig = plt.figure(1,figsize=(12,6))
    plt.style.use('seaborn')
    ax = fig.add_subplot(111)
    def animate(i):
        ax.cla()
        ax.grid()
        barhs = []
        barhs += ax.barh(Region_list[i],Confiremd_list[i],color=[l for l in Color_list],label=Date_list[i])
        plt.legend(loc='best')
        for a,b in zip(Region_list[i],Confiremd_list[i]):
            plt.text(b+0.5,a,'%.0f'%b,va='center')
        plt.xlabel('Confiremd',fontsize=14)
        plt.ylabel('Country/Region',fontsize=14)
        plt.title('Confirmed cases of Covid19 by Country/Region',fontsize=20)
        return barhs
    anim = animation.FuncAnimation(fig,animate,frames = len(Region_list),interval=50,repeat=True)
    return plt,anim

In [ ]:
Region_list = np.array(pd.DataFrame(Region_list).iloc[:,:10]).tolist()
Confiremd_list = np.array(pd.DataFrame(Confiremd_list).iloc[:,:10]).tolist()
plt,anim = draw_chart(Region_list,Confiremd_list)
anim.save('Covid-Visualization.gif')
plt.show()